# 🤖 Function Calling: Connecting LLMs to the Real World

Welcome to Exercise #13! In this notebook, you'll learn how to extend Large Language Models beyond text generation by teaching them to interact with external functions and tools.

---

## 🎯 What You'll Learn

By the end of this tutorial, you'll be able to:

1. **Understand function calling fundamentals** - What it is, why it matters, and when to use it
2. **Define tool schemas** - Create structured descriptions that help LLMs understand available functions
3. **Master the function calling loop** - Execute the complete request-response-execution-response cycle
4. **Handle responses correctly** - Distinguish between text responses and function call requests
5. **Build production-ready implementations** - Write reusable, error-handled function calling code

---

## 💼 Business Value: Why Function Calling Matters

Function calling bridges the gap between conversational AI and real-world actions. Instead of LLMs just generating text, they can now:

- **Access live data** - Fetch current weather, stock prices, database records
- **Perform actions** - Book appointments, send emails, update records
- **Query systems** - Search knowledge bases, run database queries, call APIs
- **Automate workflows** - Chain multiple operations together intelligently

### 🌟 Real-World Examples:

- **Customer Service Chatbots** - Check order status, process refunds, schedule callbacks
- **Personal Assistants** - Book meetings, set reminders, check calendars
- **Data Analysis Tools** - Query databases, generate reports, visualize data
- **Smart Home Control** - Adjust temperature, turn on lights, check security cameras
- **E-commerce** - Search products, check inventory, process orders

Without function calling, LLMs can only *suggest* what to do. With function calling, they can actually *do it*.

---

Let's get started! 🚀

---

# 📚 1. Theory: What is Function Calling?

## Core Concept

**Function calling** is a capability that allows Large Language Models to intelligently request the execution of external functions. Instead of the LLM trying to generate an answer from its training data alone, it can recognize when it needs external information or actions, and request specific function calls with appropriate parameters.

Think of it like this: Imagine you're a manager (the LLM) with a team of specialists (functions). When someone asks you a question, you can either answer from your own knowledge, or you can ask a specialist on your team to help. Function calling is how you ask your specialists for help.

## Why It Matters

LLMs are trained on static data with a knowledge cutoff. They don't have access to:
- Real-time information (current weather, stock prices, etc.)
- Private data (your database, user records, etc.)
- System capabilities (sending emails, updating databases, etc.)

Function calling solves this by letting LLMs:
1. **Recognize** when external data or actions are needed
2. **Request** specific functions with the right parameters
3. **Incorporate** the results into their responses

## When to Use Function Calling vs. Just Prompting

**Use regular prompting when:**
- The answer can be generated from the LLM's training data
- You need creative or analytical responses
- No real-time or private data is required

**Use function calling when:**
- You need real-time data (weather, prices, availability)
- You need to query private systems (databases, APIs)
- You need to perform actions (book, send, update, delete)
- You want structured, reliable data access

## The Request-Response Cycle

Here's how function calling works at a high level:

```
┌─────────────┐
│ User Query  │  "What's the weather in Paris?"
└──────┬──────┘
       │
       ▼
┌─────────────────┐
│   LLM Decides   │  "I need to call get_current_weather(location='Paris, France')"
└──────┬──────────┘
       │
       ▼
┌─────────────────┐
│ Function Call   │  Execute: get_current_weather(location='Paris, France')
└──────┬──────────┘
       │
       ▼
┌─────────────────┐
│    Execute      │  Return: {"temperature": 18, "condition": "Cloudy"}
└──────┬──────────┘
       │
       ▼
┌─────────────────┐
│  Result Back    │  Send function result back to LLM
└──────┬──────────┘
       │
       ▼
┌─────────────────┐
│  LLM Response   │  "The weather in Paris is currently 18°C and cloudy."
└─────────────────┘
```

## 💡 Key Point: Function Calling vs. Text Generation

**Important distinction:**
- **Without function calling:** The LLM would try to guess the weather based on general knowledge ("Paris typically has...")
- **With function calling:** The LLM recognizes it needs current data, requests the function, and uses the actual result

The LLM doesn't *execute* the function itself - it *requests* that you execute it. You're still in control!

---

## ✅ Key Takeaways:

- Function calling lets LLMs request execution of external functions instead of just generating text
- It bridges LLMs with real-time data, private systems, and actionable capabilities
- The LLM decides WHEN to call a function and WHICH parameters to use
- You (the developer) execute the actual function and return results to the LLM
- This enables building chatbots that can actually DO things, not just talk about them

---

# 🔧 2. Setup

Before we dive into function calling, let's set up our environment. We'll need:
1. OpenAI API access
2. The OpenAI Python library
3. A few supporting libraries

## 📦 Install Dependencies

First, let's install the required libraries:

In [ ]:
# Install the OpenAI library
!pip install -q openai

# Suppress deprecation warnings for cleaner output
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

print("✅ All dependencies installed!")

## 🔑 API Key Configuration

You have two methods to provide your OpenAI API key:

**Method 1 (Recommended)**: Use Colab Secrets
1. Click the 🔑 icon in the left sidebar
2. Click "Add new secret"
3. Name: `OPENAI_API_KEY`
4. Value: Your OpenAI API key
5. Enable notebook access

**Method 2 (Fallback)**: Manual input when prompted

Run the cell below to configure authentication:

In [ ]:
# Configure OpenAI API key
# Method 1: Try to get API key from Colab secrets (recommended)
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    # Method 2: Manual input (fallback)
    from getpass import getpass
    print("💡 To use Colab secrets: Go to 🔑 (left sidebar) → Add new secret → Name: OPENAI_API_KEY")
    OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

# Set the API key as an environment variable
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Validate that the API key is set
if not OPENAI_API_KEY or OPENAI_API_KEY.strip() == "":
    raise ValueError("❌ ERROR: No API key provided!")

print("✅ Authentication configured!")

# Configure which OpenAI model to use
OPENAI_MODEL = "gpt-5-nano"  # Cost-efficient model for function calling
print(f"🤖 Selected Model: {OPENAI_MODEL}")

## 🚀 Initialize OpenAI Client

Now let's import the necessary libraries and create our OpenAI client:

In [ ]:
# Import required libraries
from openai import OpenAI
import json
import random

# Initialize the OpenAI client
# This client will be used to make all API calls
client = OpenAI(api_key=OPENAI_API_KEY)

print("✅ OpenAI client initialized successfully!")
print("\n🎓 Ready to learn about function calling!")

---

# 📋 3. Understanding Tool Schemas

## What is a Tool Schema?

A **tool schema** is a JSON specification that describes a function to the LLM. Think of it as a menu that tells the LLM:
- What functions are available
- What each function does
- What parameters each function accepts
- What types those parameters should be
- Which parameters are required

## Why We Need Structured Schemas

The LLM needs to understand:
1. **When** to use each function (based on the description)
2. **How** to use it (what parameters to provide)
3. **What** values are acceptable (types, enums, etc.)

Without a clear schema, the LLM wouldn't know which function to call or how to call it correctly.

## 💡 Key Point: Schema vs. Implementation

**Critical distinction:**
- **Tool Schema (JSON):** A *description* of the function for the LLM to read
- **Function Implementation (Python):** The *actual code* that executes when called

The schema is like a restaurant menu - it describes the dish. The implementation is like the kitchen - it actually makes the dish.

The LLM only sees the schema. You execute the implementation.

---

## 🌤️ Step 1: Define the Actual Functions (Implementation)

Let's start by creating our actual Python functions. For this tutorial, we'll use mock weather functions that simulate API calls:

In [ ]:
# IMPLEMENTATION: These are the actual Python functions
# The LLM will REQUEST these functions, and we will EXECUTE them

def get_current_weather(location: str, format: str) -> dict:
    """
    Mock function to simulate getting the current weather.
    In a real application, this would call an actual weather API.
    
    Args:
        location (str): The city and state, e.g., San Francisco, CA.
        format (str): The temperature format, either 'celsius' or 'fahrenheit'.
    
    Returns:
        dict: A mock response with random weather data.
    """
    # Generate random weather data for demonstration
    temperature = random.uniform(15.0, 35.0) if format == "celsius" else random.uniform(60.0, 95.0)
    weather_conditions = random.choice(["Sunny", "Cloudy", "Rainy", "Stormy", "Windy"])
    
    return {
        "location": location,
        "temperature": round(temperature, 2),
        "unit": format,
        "condition": weather_conditions
    }


def get_n_day_weather_forecast(location: str, format: str, num_days: int) -> dict:
    """
    Mock function to simulate getting an N-day weather forecast.
    In a real application, this would call an actual weather API.
    
    Args:
        location (str): The city and state, e.g., San Francisco, CA.
        format (str): The temperature format, either 'celsius' or 'fahrenheit'.
        num_days (int): The number of days to forecast.
    
    Returns:
        dict: A mock response with random weather data for each day.
    """
    forecast = []
    for day in range(num_days):
        temperature = random.uniform(15.0, 35.0) if format == "celsius" else random.uniform(60.0, 95.0)
        weather_conditions = random.choice(["Sunny", "Cloudy", "Rainy", "Stormy", "Windy"])
        forecast.append({
            "day": f"Day {day + 1}",
            "temperature": round(temperature, 2),
            "unit": format,
            "condition": weather_conditions
        })
    
    return {
        "location": location,
        "forecast": forecast
    }

print("✅ Weather functions defined!")
print("\n📝 Note: These are MOCK functions - they generate random data for learning purposes.")
print("   In production, these would call real weather APIs.")

Let's quickly test our functions to see how they work:

In [ ]:
# Test the current weather function
test_result = get_current_weather("Paris, France", "celsius")
print("🧪 Test: get_current_weather('Paris, France', 'celsius')")
print(json.dumps(test_result, indent=2))

print("\n" + "="*50 + "\n")

# Test the forecast function
test_forecast = get_n_day_weather_forecast("Tokyo, Japan", "celsius", 3)
print("🧪 Test: get_n_day_weather_forecast('Tokyo, Japan', 'celsius', 3)")
print(json.dumps(test_forecast, indent=2))

## 📜 Step 2: Define Tool Schemas (Descriptions for the LLM)

Now that we have our actual functions, we need to describe them to the LLM using JSON schemas. The LLM will read these descriptions to understand when and how to call our functions.

Let's create the tool schemas:

In [ ]:
# SCHEMAS: These are descriptions that tell the LLM about our functions
# The LLM reads these to decide when and how to call functions

tools = [
    {
        # Type: Specifies this is a function tool
        "type": "function",

        # Name: Must match the actual Python function name EXACTLY
        "name": "get_current_weather",

        # Description: Tells the LLM WHEN to use this function
        # Be clear and specific - this guides the LLM's decision-making
        "description": "Get the current weather in a given location. Use this when the user asks about current or present weather conditions.",

        # Parameters: JSON Schema describing the function inputs
        "parameters": {
            "type": "object",

            # Properties: Each parameter and its details
            "properties": {
                "location": {
                    "type": "string",
                    # Description helps LLM know what format to use
                    "description": "The city and state, e.g., San Francisco, CA or Paris, France"
                },
                "format": {
                    "type": "string",
                    # Enum restricts to specific valid values
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Use 'celsius' for metric and 'fahrenheit' for imperial."
                }
            },

            # Required: List of mandatory parameters
            # The LLM will try to gather these before calling the function
            "required": ["location", "format"]
        }
    },
    {
        "type": "function",

        # Second function: Weather forecast
        "name": "get_n_day_weather_forecast",

        # Clear description distinguishes this from get_current_weather
        "description": "Get an N-day weather forecast for a given location. Use this when the user asks about future weather or multi-day forecasts.",

        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g., San Francisco, CA or Paris, France"
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Use 'celsius' for metric and 'fahrenheit' for imperial."
                },
                "num_days": {
                    "type": "integer",
                    "description": "The number of days to forecast, between 1 and 7 days"
                }
            },
            "required": ["location", "format", "num_days"]
        }
    }
]

print("✅ Tool schemas defined!")
print(f"\n📋 Number of tools available to LLM: {len(tools)}")
print("\n🔍 Tool names:")
for tool in tools:
    print(f"  - {tool['name']}")

## 🔍 Schema Deep Dive

Let's break down what each part of the schema does:

### Structure Breakdown

```json
{
  "type": "function",  // Tells OpenAI this is a function tool
  "name": "get_current_weather",  // Must match Python function name
  "description": "...",  // Helps LLM decide WHEN to use this
  "parameters": {  // JSON Schema format
      "type": "object",
      "properties": {  // Define each parameter
        "location": {
          "type": "string",  // Data type
          "description": "..."  // Helps LLM know what to provide
        },
        "format": {
          "type": "string",
          "enum": ["celsius", "fahrenheit"]  // Restricts to valid values
        }
      },
      "required": ["location", "format"]  // Mandatory parameters
    }
  }
}
```

### Key Components Explained

1. **`type: "function"`** - Tells the LLM this is a function it can call

2. **`name`** - The function identifier. **Must match your Python function name exactly!**

3. **`description`** - Critical for helping the LLM decide when to use this function. Be specific about:
   - What the function does
   - When it should be used
   - How it differs from similar functions

4. **`parameters`** - Uses JSON Schema format to define inputs:
   - `type: "object"` - Parameters are provided as an object
   - `properties` - Each parameter's definition
   - Each property has a `type` and `description`

5. **`required`** - Array of mandatory parameter names. The LLM will try to gather these before calling.

6. **`enum`** - Restricts parameter values to a specific list. Helps prevent invalid inputs.

### ⚠️ Common Mistake: Mismatched Names

**Wrong:**
```python
# Python function
def get_weather(location, format):
    ...

# Schema
{
    "name": "getCurrentWeather",  # ❌ Doesn't match!
    ...
}
```

**Correct:**
```python
# Python function
def get_weather(location, format):
    ...

# Schema
{
    "name": "get_weather",  # ✅ Exact match!
    ...
}
```

---

## ✅ Key Takeaways: Tool Schemas

- Tool schemas are JSON descriptions that tell the LLM about available functions
- Schemas are NOT the functions themselves - they're instructions for the LLM
- The `name` field must match your Python function name exactly
- The `description` field is critical - it helps the LLM decide when to use the function
- Parameter descriptions guide the LLM on what values to provide
- Use `enum` to restrict parameters to specific valid values
- The `required` array tells the LLM which parameters are mandatory

---

# 🔄 4. Understanding LLM Responses

## Response Types

When you make an API call with function calling enabled, the LLM can respond in two ways:

1. **Regular text response** - The LLM provides a conversational answer (no function needed)
2. **Function call request** - The LLM requests to execute one or more functions

Understanding which type of response you received is crucial for handling function calling correctly.

## Response Structure

The response object contains:
- **`content`** - Text content (if the LLM is providing a conversational response)
- **`tool_calls`** - List of function calls (if the LLM wants to call functions)

We need to check which one is present to know how to proceed.

## 💡 Key Point: LLMs Can Ask for Clarification

The LLM won't just blindly call functions. If it doesn't have enough information, it will ask the user for clarification. This is intelligent behavior!

For example:
- User: "What's the weather?"
- LLM: "I'd be happy to check the weather for you. Which city are you interested in?"

---

## 🧪 Example 1: LLM Asks for Clarification

Let's see what happens when we ask a vague question:

In [ ]:
# Start a new conversation
# In function calling, we maintain a conversation history with messages
messages = [
    {
        "role": "system",
        "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    },
    {
        "role": "user",
        "content": "What's the weather like today?"
    }
]

# Make API call with function calling enabled
# The 'tools' parameter tells the LLM what functions are available
response = client.responses.create(
    model=OPENAI_MODEL,
    input=messages,
    tools=tools,
    tool_choice="auto"  # Let the LLM decide whether to call a function
)

# Extract the assistant's message
# Get function calls from response
function_calls = [item for item in response.output if hasattr(item, 'type') and item.type == 'function_call']

print("🤖 Assistant Response:")
print("="*60)

# Check if the response contains tool calls or regular content
if function_calls:
    print("📞 Response type: FUNCTION CALL")
    print(f"Number of function calls: {len(function_calls)}")
    for tool_call in function_calls:
        print(f"  - Function: {tool_call.name}")
else:
    print("💬 Response type: TEXT CONTENT")
    print(f"\nContent: {response.output_text}")

print("="*60)

### 🔍 What Just Happened?

The LLM received our question "What's the weather like today?" but noticed it's **missing required information**:
- We didn't specify a location
- We didn't specify celsius or fahrenheit

Instead of making assumptions, the LLM asked for clarification. This is the **right behavior** - the system prompt told it not to guess!

The response contains **`content`** (text), not **`tool_calls`** (function requests).

---

## 🧪 Example 2: LLM Calls a Function

Now let's provide complete information:

In [ ]:
# Continue the conversation by adding the user's clarification
# We're building on the previous messages - this is ONE conversation
# Note: For Responses API, we add function calls via tool messages  # Add the assistant's clarification question
messages.append({
    "role": "user",
    "content": "I'm in Glasgow, Scotland and prefer Celsius."
})

# Make another API call with the updated conversation
response = client.responses.create(
    model=OPENAI_MODEL,
    input=messages,
    tools=tools,
    tool_choice="auto"
)

# Get function calls from response
function_calls = [item for item in response.output if hasattr(item, 'type') and item.type == 'function_call']

print("🤖 Assistant Response:")
print("="*60)

# Check the response type
if function_calls:
    print("📞 Response type: FUNCTION CALL")
    print(f"\nThe LLM decided to call a function!\n")
    print(f"Number of function calls: {len(function_calls)}")
    
    for i, tool_call in enumerate(function_calls, 1):
        print(f"\nFunction Call #{i}:")
        print(f"  - Function name: {tool_call.name}")
        print(f"  - Arguments: {tool_call.arguments}")
        print(f"  - Tool call ID: {tool_call.id}")
else:
    print("💬 Response type: TEXT CONTENT")
    print(f"\nContent: {response.output_text}")

print("="*60)

## 📊 Anatomy of a Tool Call Response

Perfect! Now the LLM has all the information it needs, so it requested a function call. Let's break down the response structure:

### Tool Call Components:

1. **`tool_calls`** - A list containing function call requests (can be multiple)

2. **`tool_calls[0].id`** - Unique identifier for this specific function call
   - Used to track which function result corresponds to which request
   - Critical for the response loop

3. **`tool_calls[0].type`** - Should always be `"function"` for function calls

4. **`tool_calls[0].function.name`** - The name of the function to call
   - This tells us WHICH function to execute
   - Will match one of the names in our tool schemas

5. **`tool_calls[0].function.arguments`** - A JSON string containing the parameters
   - ⚠️ Important: This is a STRING, not a dict!
   - We need to parse it with `json.loads()` before using it

Let's extract these components:

In [ ]:
# Extract the tool call information
# We're accessing the FIRST tool call (index 0)
tool_call = function_calls[0]

print("🔍 Extracting Tool Call Information:")
print("="*60)

# 1. Tool call ID (for tracking)
tool_call_id = tool_call.call_id
print(f"\n1️⃣ Tool Call ID: {tool_call_id}")
print("   (This unique ID tracks this specific function call)")

# 2. Function name (which function to call)
function_name = tool_call.name
print(f"\n2️⃣ Function Name: {function_name}")
print("   (This tells us WHICH function to execute)")

# 3. Arguments (the parameters - comes as a JSON string)
function_arguments_string = tool_call.arguments
print(f"\n3️⃣ Arguments (as JSON string): {function_arguments_string}")
print(f"   Type: {type(function_arguments_string)}")

# 4. Parse the arguments from JSON string to dictionary
function_arguments = json.loads(function_arguments_string)
print(f"\n4️⃣ Arguments (parsed to dict): {function_arguments}")
print(f"   Type: {type(function_arguments)}")
print(f"\n   Location: {function_arguments.get('location')}")
print(f"   Format: {function_arguments.get('format')}")

print("\n" + "="*60)
print("\n✅ Now we know WHAT function to call and WITH WHAT parameters!")

## 💡 Key Point: Arguments Are JSON Strings

**Important detail:**
- `tool_call.function.arguments` is a **JSON string**, not a Python dict
- You must parse it with `json.loads()` before using it
- This is easy to forget and causes errors!

```python
# ❌ Wrong - arguments is a string
location = tool_call.function.arguments['location']  # Error!

# ✅ Correct - parse first, then use
args = json.loads(tool_call.function.arguments)
location = args['location']  # Works!
```

---

## ✅ Key Takeaways: LLM Responses

- LLM responses can contain either text content OR function calls (or both)
- Always check if `tool_calls` exists before trying to access it
- LLMs will ask for clarification if they don't have required information (smart behavior!)
- Each tool call has: `id` (tracking), `function.name` (which function), `function.arguments` (parameters)
- Function arguments come as JSON strings - you must parse them with `json.loads()`
- The `tool_call_id` is critical - you'll need it when sending the function result back

---

# 🔁 5. The Complete Function Calling Loop

## Understanding the Full Cycle

Now we're ready to see the complete function calling flow. This is where everything comes together!

### The 5-Step Process:

```
STEP 1: Initial API Call
├─ Send user query to LLM with available tools
└─ LLM responds with function call request

STEP 2: Extract Function Details
├─ Get tool_call_id (for tracking)
├─ Get function name (which function to call)
└─ Parse arguments (what parameters to use)

STEP 3: Execute the Function
├─ Match function name to actual Python function
├─ Call the function with extracted arguments
└─ Get the function result

STEP 4: Send Result Back to LLM
├─ Create a "user" message with the function result
├─ Format the result clearly for the LLM
└─ Append to conversation history

STEP 5: Get Final Response
├─ Make another API call with updated conversation
├─ LLM uses function result to create natural language response
└─ Return final answer to user
```

## 💡 Key Point: This is ONE Conversation

**Critical concept:**
- We're NOT starting a new conversation for each step
- We're CONTINUING the same conversation by appending messages
- The messages list grows: User → Assistant (tool call) → Tool (result) → Assistant (final answer)

Think of it like a real conversation:
1. User: "What's the weather?"
2. Assistant: *checks weather* (function call)
3. System: *weather data* (function result)
4. Assistant: "It's 22°C and sunny!" (final response)



## 💡 Key Point: Responses API Difference

**Important for gpt-5-nano (Responses API):**
- The Responses API does NOT support the `'tool'` role
- Supported roles: `'assistant'`, `'system'`, `'developer'`, and `'user'`
- Function results must be sent as **'user'** messages
- Format: `{"role": "user", "content": f"Function {name} returned: {result}"}`

This is different from the Chat Completions API which uses a dedicated `'tool'` role.
## ⚠️ Common Mistake: Creating New Conversations

**Wrong approach:**
```python
# ❌ Starting fresh conversations
response1 = client.responses.create(input=[{"role": "user", "content": "Weather?"}])
# ... execute function ...
response2 = client.responses.create(input=[{"role": "user", "content": result}])  # Wrong!
```

**Correct approach:**
```python
# ✅ Continuing the same conversation
messages = [{"role": "user", "content": "Weather?"}]
response1 = client.responses.create(input=messages)
messages.append(response1.output)  # Add assistant's function call
messages.append(tool_result)  # Add function result
response2 = client.responses.create(input=messages)  # Continue conversation
```

---

## 🎯 Let's Execute the Complete Loop!

We'll go through each step clearly with detailed explanations.

### 🔷 STEP 1: Initial API Call

In [ ]:
print("🔷 STEP 1: Making initial API call with function calling enabled")
print("="*60)

# Start fresh conversation with a clear, complete query
messages = [
    {
        "role": "system",
        "content": "You are a helpful weather assistant. Use the provided functions to get accurate weather data."
    },
    {
        "role": "user",
        "content": "What's the weather in Bratislava right now? Use Celsius."
    }
]

print("📤 Sending request to LLM...")
print(f"   User query: '{messages[1]['content']}'")
print(f"   Available tools: {len(tools)}")

# Make the API call with tools enabled
response = client.responses.create(
    model=OPENAI_MODEL,
    input=messages,
    tools=tools,
    tool_choice="auto"  # Let LLM decide
)

# Store the assistant's message (which will contain the function call)
# Get function calls from response
function_calls = [item for item in response.output if hasattr(item, 'type') and item.type == 'function_call']

print("\n📥 Response received!")
print(f"   Response type: {'FUNCTION CALL' if function_calls else 'TEXT'}")

if function_calls:
    print(f"   Function to call: {function_calls[0].name}")

print("\n✅ Step 1 complete!")

### 🔷 STEP 2: Extract Tool Call Details

In [ ]:
print("🔷 STEP 2: Extracting function call details")
print("="*60)

# Access the first (and in this case, only) tool call
tool_call = function_calls[0]

# Extract the three key pieces of information
tool_call_id = tool_call.call_id
function_name = tool_call.name
function_arguments = json.loads(tool_call.arguments)

print("📋 Extracted information:")
print(f"   Tool Call ID: {tool_call_id}")
print(f"   Function Name: {function_name}")
print(f"   Function Arguments: {json.dumps(function_arguments, indent=6)}")

print("\n✅ Step 2 complete!")

### 🔷 STEP 3: Execute the Actual Function

In [ ]:
print("🔷 STEP 3: Executing the actual Python function")
print("="*60)

# Match the function name to the actual Python function and execute it
# We use if/elif to route to the correct function
if function_name == "get_current_weather":
    print(f"🌤️  Calling: get_current_weather()")
    print(f"   Parameters: location='{function_arguments['location']}', format='{function_arguments['format']}'")
    
    # Execute the function with the extracted arguments
    # The ** operator unpacks the dictionary as keyword arguments
    function_result = get_current_weather(**function_arguments)
    
elif function_name == "get_n_day_weather_forecast":
    print(f"📅 Calling: get_n_day_weather_forecast()")
    print(f"   Parameters: {function_arguments}")
    function_result = get_n_day_weather_forecast(**function_arguments)
else:
    # This shouldn't happen if our schemas are correct, but good to handle
    print(f"❌ Unknown function: {function_name}")
    function_result = {"error": "Unknown function"}

print("\n📊 Function result:")
print(json.dumps(function_result, indent=3))

print("\n✅ Step 3 complete!")

### 🔷 STEP 4: Add Function Result to Conversation

In [ ]:
print("🔷 STEP 4: Adding function result to conversation history")
print("="*60)

# First, append the assistant's message (the function call request)
# This is important for maintaining conversation context!
# Note: With Responses API, we don't need to append the function call request
# We only append the tool message with results

# Now, create a "tool" message with the function result
# This tells the LLM "here's the result of the function you requested"
tool_message = {
    "role": "user",  # Responses API requires 'user' role for function results
    "content": f"Function {function_name} returned: {json.dumps(function_result)}"
}

# Append the tool message to the conversation
messages.append(tool_message)
print("✅ Appended function result to messages")

print("\n📝 Current conversation structure:")
for i, msg in enumerate(messages, 1):
    role = msg.get('role', 'N/A')
    if role == 'tool':
        print(f"   {i}. {role.upper()} - Function result from {msg['name']}")
    # (function calls are handled via tool messages)
    else:
        content_preview = str(msg.get('content', ''))[:50]
        print(f"   {i}. {role.upper()} - {content_preview}...")

print("\n✅ Step 4 complete!")

### 🔷 STEP 5: Get Final Response from LLM

In [ ]:
print("🔷 STEP 5: Getting final natural language response")
print("="*60)

# Make another API call with the updated conversation
# Now the conversation includes the function result!
# This time we DON'T pass 'tools' because we don't want another function call
print("📤 Sending conversation (with function result) back to LLM...")

final_response = client.responses.create(
    model=OPENAI_MODEL,
    input=messages
    # Note: No 'tools' parameter - we're done with function calling
)

# Extract the final response
final_message = final_response.output_text

print("\n📥 Final response received!\n")
print("="*60)
print("🤖 ASSISTANT:")
print(final_message)
print("="*60)

print("\n✅ Step 5 complete!")
print("\n🎉 FUNCTION CALLING LOOP COMPLETE!")

## 🎯 What Just Happened? Complete Flow Recap

Let's trace through the entire conversation flow:

### Message Flow:

1. **User Message** (Step 1)
   - "What's the weather in Bratislava right now? Use Celsius."

2. **Assistant Message - Function Call Request** (Step 1 response)
   - "I need to call `get_current_weather` with `location='Bratislava, Slovakia'` and `format='celsius'`"

3. **We Execute the Function** (Steps 2-3)
   - Extract function name and parameters
   - Call the actual Python function
   - Get result: `{"location": "Bratislava", "temperature": 22.5, "condition": "Sunny"}`

4. **User Message - Function Result** (Step 4)
   - Send the function result back to the LLM as a user message
   - Formatted as: "Function {name} returned: {result}"

5. **Assistant Message - Final Response** (Step 5)
   - LLM uses the function result to create natural language response
   - "The current weather in Bratislava is 22.5°C and sunny!"

### Why This Works:

- **Conversation continuity:** We maintained one conversation thread throughout
- **Proper message roles:** System → User → Assistant (tool call) → Tool (result) → Assistant (answer)
- **Tool call tracking:** The `tool_call_id` linked the function result to the original request
- **Context preservation:** Each API call had access to the full conversation history

---

## ✅ Key Takeaways: Function Calling Loop

- Function calling is a 5-step process: Request → Extract → Execute → Return → Respond
- This is ONE continuous conversation, not separate interactions
- Always append messages to maintain conversation history
- The assistant message (with function call) AND the tool message (with result) both get added
- The `tool_call_id` is critical for linking results to requests
- The final API call doesn't include `tools` parameter (we don't want more function calls)
- The LLM uses the function result to create a natural, conversational response

---

# 🔧 6. Helper Function for Reusability

Now that we understand the complete flow, let's wrap it into a reusable function. This makes function calling much easier to use in real applications!

## Why Create a Helper Function?

Instead of manually coding the 5-step process every time, we can:
- Encapsulate the logic once
- Handle errors gracefully
- Make it easier to use function calling throughout our application
- Support multiple function calls in one response

Let's create a robust helper function:

In [ ]:
def chat_with_function_execution(messages, tools, available_functions):
    """
    Execute a chat completion with automatic function calling.
    
    This function handles the complete function calling loop:
    1. Makes initial API call
    2. If LLM requests function calls, executes them
    3. Sends results back to LLM
    4. Returns final response
    
    Args:
        messages (list): Conversation history
        tools (list): Available tool schemas
        available_functions (dict): Mapping of function names to actual Python functions
    
    Returns:
        tuple: (final_response_text, updated_messages)
    """
    
    print("🤖 Starting chat with function execution...\n")
    
    # STEP 1: Initial API call
    print("📤 Step 1: Making initial API call...")
    response = client.responses.create(
        model=OPENAI_MODEL,
        input=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    # Get function calls from response
    function_calls = [item for item in response.output if hasattr(item, 'type') and item.type == 'function_call']
    
    # Check if LLM wants to call functions
    if not function_calls:
        # No function calls - just return the text response
        print("💬 No function calls needed. Returning text response.\n")
        # Note: For Responses API, we add function calls via tool messages
        return response.output_text, messages
    
    # LLM wants to call one or more functions
    print(f"📞 LLM requested {len(function_calls)} function call(s)\n")
    
    # Append the assistant's function call request
    # Note: For Responses API, we add function calls via tool messages
    
    # STEPS 2-4: Process each function call
    for i, tool_call in enumerate(function_calls, 1):
        # STEP 2: Extract function details
        function_name = tool_call.name
        function_arguments = json.loads(tool_call.arguments)
        tool_call_id = tool_call.call_id
        
        print(f"🔧 Function call {i}: {function_name}")
        print(f"   Arguments: {function_arguments}")
        
        # STEP 3: Execute the function
        try:
            # Check if function exists in our available functions
            if function_name not in available_functions:
                function_result = {
                    "error": f"Function '{function_name}' not found"
                }
                print(f"   ❌ Error: Function not found")
            else:
                # Execute the function
                function_to_call = available_functions[function_name]
                function_result = function_to_call(**function_arguments)
                print(f"   ✅ Function executed successfully")
        
        except Exception as e:
            # Handle execution errors gracefully
            function_result = {
                "error": f"Function execution failed: {str(e)}"
            }
            print(f"   ❌ Error: {str(e)}")
        
        # STEP 4: Add function result to conversation
        messages.append({
            "role": "user",
            "content": f"Function {function_name} returned: {json.dumps(function_result)}"
        })
        print(f"   📝 Result added to conversation\n")
    
    # STEP 5: Get final response with function results
    print("📤 Step 5: Getting final response from LLM...\n")
    final_response = client.responses.create(
        model=OPENAI_MODEL,
        input=messages
    )
    
    final_message = final_response.output_text
    
    print("✅ Function calling complete!\n")
    
    return final_message, messages


print("✅ Helper function created!")
print("\n📝 Usage:")
print("   response, updated_messages = chat_with_function_execution(messages, tools, available_functions)")

## 🧪 Test the Helper Function

Let's test our helper function with a multi-day forecast request:

In [ ]:
# Create a dictionary mapping function names to actual Python functions
# This makes it easy for our helper to route function calls
available_functions = {
    "get_current_weather": get_current_weather,
    "get_n_day_weather_forecast": get_n_day_weather_forecast
}

# Start a new conversation
test_messages = [
    {
        "role": "system",
        "content": "You are a helpful weather assistant. Use the provided functions to get accurate weather data."
    },
    {
        "role": "user",
        "content": "What's the 5-day forecast for San Francisco in Fahrenheit?"
    }
]

print("🧪 Testing helper function with forecast request")
print("="*60)
print(f"User query: {test_messages[1]['content']}\n")
print("="*60 + "\n")

# Use our helper function!
response, updated_messages = chat_with_function_execution(
    messages=test_messages,
    tools=tools,
    available_functions=available_functions
)

print("="*60)
print("🤖 FINAL RESPONSE:")
print("="*60)
print(response)
print("="*60)

### 🎉 Success!

Our helper function:
- ✅ Detected the LLM wanted to call `get_n_day_weather_forecast`
- ✅ Extracted the parameters (location, format, num_days)
- ✅ Executed the actual Python function
- ✅ Sent the result back to the LLM
- ✅ Got a natural language response

All in one simple function call!

---

# ⚙️ 7. Understanding the `tool_choice` Parameter

## What is `tool_choice`?

The `tool_choice` parameter gives you control over **whether and how** the LLM uses function calling. It tells the LLM:
- Can you call functions?
- Should you call functions?
- Must you call functions?

## Available Options

### 1. `tool_choice="auto"` (Default)
- **Behavior:** LLM decides whether to call a function or respond with text
- **When to use:** Most of the time! This gives the LLM freedom to choose the best approach
- **Example:** User asks "What's the weather?" → LLM calls function. User asks "Tell me a joke" → LLM responds with text

### 2. `tool_choice="none"`
- **Behavior:** LLM will NOT call any functions, only text responses
- **When to use:** When you want to disable function calling temporarily
- **Example:** After getting function results, you don't want another function call

### 3. `tool_choice="required"`
- **Behavior:** LLM MUST call at least one function (cannot respond with text only)
- **When to use:** When you need to force a function call, even if the LLM might normally just answer with text
- **Example:** Data extraction tasks where you always need structured output

### 4. `tool_choice={"type": "function", "name": "function_name"}`
- **Behavior:** Forces the LLM to call a SPECIFIC function
- **When to use:** When you know exactly which function should be called
- **Example:** User clicked a "Get Weather" button - you want to force `get_current_weather`

## 💡 Key Point: When to Force Function Calls

Forcing function calls is useful when:
- Building structured data extraction tools
- Creating button-driven interfaces ("Check Weather" button = force weather function)
- Ensuring consistent behavior for specific commands
- Testing functions during development

---

## 🧪 Example: `tool_choice="none"`

Let's see what happens when we disable function calling:

In [ ]:
print("🧪 Testing with tool_choice='none'")
print("="*60)

messages = [
    {
        "role": "user",
        "content": "What's the weather in Paris right now?"
    }
]

# Make API call with tool_choice="none" - LLM CANNOT call functions
response = client.responses.create(
    model=OPENAI_MODEL,
    input=messages,
    tools=tools,
    tool_choice="none"  # Disable function calling
)

print("🤖 Assistant response (tool_choice='none'):")
print(response.output_text)
print("\n" + "="*60)
print("\n📝 Notice: The LLM responded with text instead of calling the weather function!")
print("   Even though the function would provide accurate data, tool_choice='none' prevented it.")

## 🧪 Example: `tool_choice="required"`

Now let's force the LLM to call a function:

In [ ]:
print("🧪 Testing with tool_choice='required'")
print("="*60)

messages = [
    {
        "role": "user",
        "content": "I'm in London and want to know if I need an umbrella. Use Celsius."
    }
]

# Make API call with tool_choice="required" - LLM MUST call a function
response = client.responses.create(
    model=OPENAI_MODEL,
    input=messages,
    tools=tools,
    tool_choice="required"  # Force function calling
)

# Get function calls from response
function_calls = [item for item in response.output if hasattr(item, 'type') and item.type == 'function_call']

if function_calls:
    print("✅ LLM was forced to call a function!")
    print(f"\nFunction called: {function_calls[0].name}")
    print(f"Arguments: {function_calls[0].arguments}")
else:
    print("❌ Unexpected: No function call despite tool_choice='required'")

print("\n" + "="*60)
print("\n📝 Notice: Even though the user didn't explicitly ask for weather,")
print("   tool_choice='required' forced the LLM to call a function to answer.")

## 🧪 Example: Force a Specific Function

Let's force the LLM to call a specific function:

In [ ]:
print("🧪 Testing with tool_choice forcing specific function")
print("="*60)

messages = [
    {
        "role": "user",
        "content": "I need weather info for Tokyo in Celsius"
    }
]

# Force the LLM to call get_current_weather specifically
response = client.responses.create(
    model=OPENAI_MODEL,
    input=messages,
    tools=tools,
    tool_choice={
        "type": "function",
        "name": "get_current_weather"
    }
)

# Get function calls from response
function_calls = [item for item in response.output if hasattr(item, 'type') and item.type == 'function_call']

if function_calls:
    print("✅ LLM called the forced function!")
    print(f"\nFunction called: {function_calls[0].name}")
    print(f"Arguments: {function_calls[0].arguments}")
    print("\n📝 Even if the user had asked for a forecast, this would call get_current_weather")
    print("   because we forced that specific function.")

print("\n" + "="*60)

---

## ✅ Key Takeaways: tool_choice Parameter

- `tool_choice="auto"` - Default, LLM decides (use this most of the time)
- `tool_choice="none"` - Disable function calling (useful after getting function results)
- `tool_choice="required"` - LLM must call at least one function
- Specific function forcing - Useful for button-driven interfaces and structured extraction
- Generally, "auto" is the best choice - let the LLM be intelligent about when to call functions

---

# 🛡️ 8. Error Handling & Edge Cases

## Why Error Handling Matters

In production applications, many things can go wrong:
- The LLM might request a function that doesn't exist
- Function arguments might be invalid or malformed
- The function itself might throw an error
- The LLM might generate invalid JSON
- Network issues might cause API failures

Robust error handling ensures your application doesn't crash and provides useful feedback.

## ⚠️ Common Mistakes to Avoid

### 1. Not Checking if `tool_calls` Exists

**Wrong:**
```python
# ❌ This will crash if no tool calls
tool_call = response.output.tool_calls[0]
```

**Correct:**
```python
# ✅ Always check first
if hasattr(response.output, 'tool_calls') and response.output.tool_calls:
    tool_call = response.output.tool_calls[0]
```

### 2. Not Handling JSON Parse Errors

**Wrong:**
```python
# ❌ Will crash if JSON is invalid
args = json.loads(tool_call.function.arguments)
```

**Correct:**
```python
# ✅ Handle parse errors
try:
    args = json.loads(tool_call.function.arguments)
except json.JSONDecodeError as e:
    print(f"Error parsing arguments: {e}")
    args = {}
```

### 3. Not Validating Function Exists

**Wrong:**
```python
# ❌ Will crash if function doesn't exist
result = available_functions[function_name](**args)
```

**Correct:**
```python
# ✅ Check function exists
if function_name in available_functions:
    result = available_functions[function_name](**args)
else:
    result = {"error": f"Function {function_name} not found"}
```

### 4. Forgetting to Convert Function Result to String

**Wrong:**
```python
# ❌ Tool message content must be a string
tool_message = {
    "role": "tool",
    "content": function_result  # ❌ This is a dict!
}
```

**Correct:**
```python
# ✅ Convert to JSON string
tool_message = {
    "role": "tool",
    "content": json.dumps(function_result)  # ✅ String
}
```

---

## 🔧 Robust Function Execution Handler

Let's create an improved function handler with comprehensive error handling:

In [ ]:
def execute_function_call_safely(tool_call, available_functions):
    """
    Safely execute a function call with comprehensive error handling.
    
    Args:
        tool_call: The tool call object from LLM response
        available_functions (dict): Mapping of function names to Python functions
    
    Returns:
        dict: Contains 'success' boolean and either 'result' or 'error'
    """
    
    try:
        # Extract function name
        function_name = tool_call.name
        
        # Check if function exists
        if function_name not in available_functions:
            return {
                "success": False,
                "error": f"Function '{function_name}' not found in available functions",
                "error_type": "function_not_found"
            }
        
        # Parse arguments with error handling
        try:
            function_arguments = json.loads(tool_call.arguments)
        except json.JSONDecodeError as e:
            return {
                "success": False,
                "error": f"Invalid JSON in function arguments: {str(e)}",
                "error_type": "json_parse_error"
            }
        
        # Execute the function with error handling
        try:
            function_to_call = available_functions[function_name]
            result = function_to_call(**function_arguments)
            
            return {
                "success": True,
                "result": result
            }
        
        except TypeError as e:
            # Wrong arguments provided
            return {
                "success": False,
                "error": f"Invalid arguments for function '{function_name}': {str(e)}",
                "error_type": "invalid_arguments"
            }
        
        except Exception as e:
            # Function execution error
            return {
                "success": False,
                "error": f"Function '{function_name}' execution failed: {str(e)}",
                "error_type": "execution_error"
            }
    
    except Exception as e:
        # Catch-all for unexpected errors
        return {
            "success": False,
            "error": f"Unexpected error: {str(e)}",
            "error_type": "unknown_error"
        }


print("✅ Robust function execution handler created!")
print("\n🛡️ This handler protects against:")
print("   - Function not found errors")
print("   - JSON parsing errors")
print("   - Invalid argument errors")
print("   - Function execution errors")
print("   - Unexpected errors")

## 🧪 Testing Error Handling

Let's test our robust handler with different error scenarios:

In [ ]:
# Create a mock tool call for testing
class MockToolCall:
    def __init__(self, function_name, arguments_json):
        self.function = type('obj', (object,), {
            'name': function_name,
            'arguments': arguments_json
        })

print("🧪 Testing error handling scenarios\n")
print("="*60)

# Test 1: Valid function call
print("\nTest 1: Valid function call")
mock_call = MockToolCall(
    "get_current_weather",
    '{"location": "Paris, France", "format": "celsius"}'
)
result = execute_function_call_safely(mock_call, available_functions)
print(f"Result: {result['success']}")
if result['success']:
    print(f"Weather data: {result['result']}")

# Test 2: Function doesn't exist
print("\n" + "-"*60)
print("\nTest 2: Non-existent function")
mock_call = MockToolCall(
    "get_stock_price",  # This function doesn't exist
    '{"symbol": "AAPL"}'
)
result = execute_function_call_safely(mock_call, available_functions)
print(f"Result: {result['success']}")
print(f"Error: {result['error']}")
print(f"Error type: {result['error_type']}")

# Test 3: Invalid JSON
print("\n" + "-"*60)
print("\nTest 3: Invalid JSON in arguments")
mock_call = MockToolCall(
    "get_current_weather",
    '{"location": "Paris", invalid json here}'
)
result = execute_function_call_safely(mock_call, available_functions)
print(f"Result: {result['success']}")
print(f"Error: {result['error']}")
print(f"Error type: {result['error_type']}")

# Test 4: Missing required argument
print("\n" + "-"*60)
print("\nTest 4: Missing required argument")
mock_call = MockToolCall(
    "get_current_weather",
    '{"location": "Paris, France"}'  # Missing 'format' argument
)
result = execute_function_call_safely(mock_call, available_functions)
print(f"Result: {result['success']}")
print(f"Error: {result['error']}")
print(f"Error type: {result['error_type']}")

print("\n" + "="*60)
print("\n✅ All error scenarios handled gracefully!")

---

## ✅ Key Takeaways: Error Handling

- Always check if `tool_calls` exists before accessing it
- Wrap JSON parsing in try/except blocks
- Validate that requested functions exist before calling them
- Handle function execution errors gracefully
- Convert function results to JSON strings for tool messages
- Return meaningful error messages that help debug issues
- Consider error types to handle different failures appropriately
- Never let unhandled exceptions crash your application

---

# 🎯 9. Complete End-to-End Example

Let's put everything together with a complete, production-ready example. This demonstrates:
- Proper conversation flow
- Error handling
- Multiple function calls
- Natural conversation

## 🌟 Complete Weather Assistant

In [ ]:
def weather_assistant(user_query):
    """
    Complete weather assistant with function calling.
    
    Args:
        user_query (str): The user's question
    
    Returns:
        str: The assistant's response
    """
    
    # Initialize conversation
    messages = [
        {
            "role": "system",
            "content": "You are a helpful weather assistant. Use the provided functions to get accurate, real-time weather data. Be friendly and informative."
        },
        {
            "role": "user",
            "content": user_query
        }
    ]
    
    # Available functions
    available_functions = {
        "get_current_weather": get_current_weather,
        "get_n_day_weather_forecast": get_n_day_weather_forecast
    }
    
    try:
        # Step 1: Initial API call
        response = client.responses.create(
            model=OPENAI_MODEL,
            input=messages,
            tools=tools,
            tool_choice="auto"
        )
        
        # Get function calls from response
        function_calls = [item for item in response.output if hasattr(item, 'type') and item.type == 'function_call']
        
        # Check if function calling is needed
        if not function_calls:
            # No function call needed - return text response
            return response.output_text
        
        # Step 2-4: Process function calls
        # Note: For Responses API, we add function calls via tool messages
        
        for tool_call in function_calls:
            # Execute function safely
            execution_result = execute_function_call_safely(tool_call, available_functions)
            
            # Prepare result for LLM
            if execution_result['success']:
                result_content = json.dumps(execution_result['result'])
            else:
                result_content = json.dumps({
                    "error": execution_result['error']
                })
            
            # Add to conversation
            messages.append({
                "role": "user",
                "content": f"Function {tool_call.name} returned: {result_content}"
            })
        
        # Step 5: Get final response
        final_response = client.responses.create(
            model=OPENAI_MODEL,
            input=messages
        )
        
        return final_response.output_text
    
    except Exception as e:
        return f"Sorry, I encountered an error: {str(e)}"


print("✅ Complete weather assistant created!")

## 🧪 Test the Complete Assistant

Let's test with different types of queries:

In [ ]:
print("🌤️  WEATHER ASSISTANT - DEMO\n")
print("="*60)

# Test 1: Current weather
print("\n👤 User: What's the current weather in London? Use Celsius.")
print("-"*60)
response = weather_assistant("What's the current weather in London? Use Celsius.")
print(f"🤖 Assistant: {response}")

# Test 2: Forecast
print("\n" + "="*60)
print("\n👤 User: Give me a 3-day forecast for New York in Fahrenheit.")
print("-"*60)
response = weather_assistant("Give me a 3-day forecast for New York in Fahrenheit.")
print(f"🤖 Assistant: {response}")

# Test 3: Vague query (should ask for clarification)
print("\n" + "="*60)
print("\n👤 User: What's the weather like?")
print("-"*60)
response = weather_assistant("What's the weather like?")
print(f"🤖 Assistant: {response}")

print("\n" + "="*60)
print("\n🎉 Complete weather assistant working perfectly!")

---

# 📚 10. Best Practices & Key Takeaways

## ✅ Best Practices for Function Calling

### 1. **Write Clear, Descriptive Function Descriptions**
   - The `description` field is critical for the LLM's decision-making
   - Be specific about WHEN to use each function
   - Distinguish similar functions clearly
   - Example: "Get the current weather" vs "Get an N-day weather forecast"

### 2. **Be Specific About Parameter Types and Constraints**
   - Use `enum` to restrict values when possible
   - Provide examples in descriptions ("e.g., San Francisco, CA")
   - Specify format requirements clearly
   - Mark required parameters appropriately

### 3. **Always Validate Function Exists Before Calling**
   - Check if function name is in your available_functions dict
   - Return meaningful error messages if not found
   - Don't let unknown functions crash your application

### 4. **Handle Errors Gracefully**
   - Wrap JSON parsing in try/except
   - Catch function execution errors
   - Return errors to the LLM in the tool message
   - Let the LLM explain errors to users naturally

### 5. **Keep Functions Focused and Simple**
   - Each function should do ONE thing well
   - Avoid complex multi-purpose functions
   - Make functions easy to test independently
   - Keep parameter lists manageable (3-5 parameters max)

### 6. **Use tool_choice Strategically**
   - Default to "auto" for natural behavior
   - Use "required" for structured data extraction
   - Use "none" when you don't want function calls
   - Force specific functions only when necessary

### 7. **Test With Various Queries**
   - Clear queries with all information
   - Vague queries missing details
   - Edge cases and error scenarios
   - Multiple function calls in one request

### 8. **Consider Cost**
   - Each function call = multiple API requests (initial + final)
   - Monitor token usage carefully
   - Cache results when appropriate
   - Use cheaper models (like gpt-5-nano) when possible

### 9. **Use Mock Functions for Learning/Testing**
   - Develop with mock data before connecting real APIs
   - Test function calling logic without external dependencies
   - Add real API calls only after logic is solid

### 10. **Maintain Conversation Context Properly**
   - Always append messages, don't create new conversations
   - Include both assistant message (tool call) and tool message (result)
   - Preserve conversation history throughout the loop

---

## 💡 Pro Tips

### 1. **Batch Function Definitions**
Create a separate file for function definitions and schemas:
```python
# functions.py
def get_weather(...):
    ...

WEATHER_TOOLS = [ ... ]
```
This keeps your main code clean and organized.

### 2. **Log Function Calls**
In production, log:
- Which functions are being called
- What parameters are being used
- Execution time and results
- Any errors that occur

This helps debug issues and understand usage patterns.

### 3. **Version Your Function Schemas**
If you change function signatures, version them:
```python
"get_weather_v2"
```
This prevents breaking changes for existing implementations.

### 4. **Add Parameter Validation**
Even though the LLM should provide valid parameters, validate them in your functions:
```python
def get_weather(location, format):
    if format not in ["celsius", "fahrenheit"]:
        raise ValueError("Invalid format")
    ...
```

---

## ⚠️ What NOT to Do

### 1. **Don't Start New Conversations**
❌ Creating a fresh message list for each API call
✅ Append to the same messages list throughout the loop

### 2. **Don't Forget to Parse Arguments**
❌ Using `tool_call.function.arguments` directly (it's a string!)
✅ Parse with `json.loads()` first

### 3. **Don't Return Non-String Content in Tool Messages**
❌ `"content": function_result` (dict)
✅ `"content": json.dumps(function_result)` (string)

### 4. **Don't Make Function Descriptions Too Vague**
❌ "Get weather" (too vague, when should this be used?)
✅ "Get the current weather in a given location. Use this for present conditions."

### 5. **Don't Ignore Errors**
❌ Letting exceptions crash your application
✅ Catch errors and handle them gracefully

---

## 🎓 Summary: What You've Learned

You now know how to:
- ✅ Understand what function calling is and why it's powerful
- ✅ Define tool schemas that describe functions to the LLM
- ✅ Distinguish between schemas (descriptions) and implementations (actual code)
- ✅ Parse and understand LLM responses (text vs tool calls)
- ✅ Execute the complete 5-step function calling loop
- ✅ Maintain conversation context properly
- ✅ Use the tool_choice parameter effectively
- ✅ Handle errors gracefully in production code
- ✅ Build reusable helper functions
- ✅ Apply best practices for robust function calling

**Congratulations!** You're now ready to build AI applications that can interact with the real world! 🎉

---

# 🏋️ 11. Practice Exercises

Now it's your turn to practice! Try these exercises to solidify your understanding:

## 📝 Exercise 1: Calculator Functions

**Goal:** Create a calculator assistant that can perform basic math operations.

**Tasks:**
1. Create four functions: `add()`, `subtract()`, `multiply()`, `divide()`
2. Each function should take two numbers as parameters
3. Create tool schemas for each function
4. Build a calculator assistant that uses these functions
5. Test with queries like:
   - "What's 15 + 27?"
   - "Divide 144 by 12"
   - "What's 8 times 9?"

**Bonus:** Add a `power()` function for exponentiation.

---

## 📝 Exercise 2: Database Query Functions

**Goal:** Create mock database query functions for a product catalog.

**Tasks:**
1. Create mock functions:
   - `search_products(query, category, max_price)`
   - `get_product_details(product_id)`
   - `check_inventory(product_id)`
2. Make functions return mock data (use random values)
3. Create tool schemas with clear descriptions
4. Build an assistant that helps users find products
5. Test with queries like:
   - "Show me laptops under $1000"
   - "What are the details for product ID 12345?"
   - "Is product 67890 in stock?"

**Bonus:** Add a `compare_products()` function that takes multiple product IDs.

---

## 💡 Tips for Exercises:

- Start with the function implementations first
- Then create the tool schemas
- Test each function independently before integrating
- Use the helper functions we created in this notebook
- Add error handling as you go
- Test with both clear and vague queries

---

## 🚀 Challenge: Multi-Step Workflow

For an advanced challenge, try creating a booking system that requires multiple function calls:

1. `check_availability(date, service_type)` - Check if time slots are available
2. `book_appointment(date, time, service_type, customer_name)` - Make a booking
3. `cancel_booking(booking_id)` - Cancel a booking

The assistant should:
- First check availability before booking
- Confirm details with the user
- Handle the booking
- Provide a booking confirmation

This requires chaining multiple function calls together intelligently!

---

# 🎓 12. Summary & Next Steps

## 🎉 Congratulations!

You've completed the Function Calling tutorial! You now have the foundational knowledge to build AI applications that can:
- Access real-time data
- Query databases and APIs
- Perform actions on behalf of users
- Integrate with existing systems

## 📚 What We Covered

1. **Theory:** Understanding what function calling is and why it matters
2. **Tool Schemas:** Defining function descriptions for the LLM
3. **Response Handling:** Parsing and understanding LLM responses
4. **Function Loop:** The complete 5-step execution cycle
5. **Helper Functions:** Building reusable, production-ready code
6. **tool_choice:** Controlling function calling behavior
7. **Error Handling:** Building robust, fault-tolerant applications
8. **Best Practices:** Professional patterns and anti-patterns

## 🚀 Next Steps

### Immediate Practice:
1. Complete the practice exercises above
2. Modify the weather assistant with your own functions
3. Build a simple chatbot for your use case

### Advanced Topics to Explore:
1. **Parallel Function Calling:** Handling multiple function calls simultaneously
2. **Function Chaining:** Using output from one function as input to another
3. **Real API Integration:** Connect to actual APIs (weather, databases, etc.)
4. **Streaming Responses:** Handle function calling with streaming
5. **State Management:** Maintain conversation state across sessions
6. **Security:** Validate and sanitize function parameters
7. **Rate Limiting:** Handle API quotas and rate limits
8. **Caching:** Cache function results for efficiency

### Related Topics:
- **Prompt Engineering:** Craft better system prompts for function calling
- **RAG (Retrieval Augmented Generation):** Combine function calling with knowledge bases
- **Agents:** Build autonomous agents that plan and execute multi-step tasks
- **Embeddings:** Use function calling with semantic search

## 📖 Resources for Further Learning

- **OpenAI Function Calling Guide:** https://platform.openai.com/docs/guides/function-calling
- **OpenAI Cookbook:** https://cookbook.openai.com
- **API Reference:** https://platform.openai.com/docs/api-reference

## 💡 Remember:

- Function calling bridges conversational AI with real-world actions
- The LLM requests functions, YOU execute them (you're in control)
- Maintain conversation context by appending to the messages list
- Always handle errors gracefully
- Start with mock functions, add real integrations later
- Test thoroughly with various query types

---

## 🙏 Thank You!

Thank you for completing this tutorial. Function calling is a powerful capability that opens up countless possibilities for AI applications. 

Keep building, keep experimenting, and most importantly - have fun creating amazing AI-powered applications! 🚀

---

**Happy coding!** 🎉